In [1]:
from random import randint, uniform
n_pop = 10
n_bits = 8
n_iter = 5
r_cross = 0.8 # crossover rate
r_mut = 0.1 # flip bits with probability

In [3]:
pop = [[randint(0,1) for x in range(n_bits)] for _ in range(n_pop)]
pop

[[0, 0, 1, 0, 0, 1, 1, 1],
 [0, 0, 1, 0, 1, 0, 0, 1],
 [0, 0, 0, 0, 1, 1, 1, 1],
 [0, 1, 1, 1, 0, 1, 1, 1],
 [1, 1, 0, 1, 0, 1, 0, 0],
 [0, 0, 1, 1, 0, 1, 1, 0],
 [0, 0, 1, 1, 1, 1, 0, 1],
 [1, 0, 1, 1, 0, 1, 0, 0],
 [0, 1, 1, 0, 0, 1, 0, 1],
 [1, 1, 0, 0, 0, 1, 1, 1]]

In [2]:
def objective(candidate):
    sum_ = 0
    for bit in candidate:
        sum_ += bit
    return sum_

In [5]:
# enumerate generations
for gen in range(n_iter):
    # evaluate all candidates in the population
    scores = [objective(c) for c in pop]
    print(scores)

[4, 3, 4, 6, 4, 4, 5, 4, 4, 5]
[4, 3, 4, 6, 4, 4, 5, 4, 4, 5]
[4, 3, 4, 6, 4, 4, 5, 4, 4, 5]
[4, 3, 4, 6, 4, 4, 5, 4, 4, 5]
[4, 3, 4, 6, 4, 4, 5, 4, 4, 5]


In [5]:
def selection(pop,scores,k=3):
    # first random selection
    selection_ix = randint(0,len(pop)-1)
    for ix in [randint(0,len(pop)-1) for x in range(k-1)]:
    # check if better (e.g. perform a tournament)
        if scores[ix] < scores[selection_ix]:
            selection_ix = ix
    return pop[selection_ix]

In [10]:
selection(pop, scores)

[0, 0, 0, 0, 1, 1, 1, 1]

In [11]:
# select parents
selected = [selection(pop, scores) for _ in range(n_pop)]
selected

[[0, 1, 1, 0, 0, 1, 0, 1],
 [0, 1, 1, 0, 0, 1, 0, 1],
 [1, 0, 1, 1, 0, 1, 0, 0],
 [0, 0, 0, 0, 1, 1, 1, 1],
 [0, 0, 1, 0, 0, 1, 1, 1],
 [0, 0, 1, 1, 0, 1, 1, 0],
 [1, 1, 0, 1, 0, 1, 0, 0],
 [1, 1, 0, 1, 0, 1, 0, 0],
 [0, 0, 1, 0, 1, 0, 0, 1],
 [0, 0, 1, 1, 0, 1, 1, 0]]

In [6]:
# crossover two parents to create two children
def crossover(p1,p2,r_cross):
    print([p1,p2])
    # children are copies of parents by default
    c1 , c2 = p1.copy(), p2.copy()
    # check for recombination
    if uniform(0,1) < r_cross:
        # select crossover point that is not on the end of the string
        pt = randint(1, len(p1)-2)
        # perform crossover
        c1 = p1[:pt] + p2[pt:]
        c2 = p2[:pt] + p1[pt:]
    return [c1,c2]

In [16]:
crossover(pop[0],pop[1],r_cross)

[[0, 0, 1, 0, 0, 1, 1, 1], [0, 0, 1, 0, 1, 0, 0, 1]]


[[0, 0, 1, 0, 0, 1, 1, 1], [0, 0, 1, 0, 1, 0, 0, 1]]

In [7]:
def mutation(bitstring, r_mut):
    for i in range(len(bitstring)):
        # check for a mutation
        if uniform(0,1) < r_mut:
            # flip the bit
            bitstring[i] = 1 - bitstring[i]

In [23]:
# create the next generation
children = list()
for i in range(0,n_pop-1,2):
    # get selected parents in pairs
    p1, p2 = selected[i], selected[i+1]
    # crossover and mutation
    for c in crossover(p1,p2,r_cross):
        # mutation
        mutation(c,r_mut)
        # store for next generation
        children.append(c)

[[0, 1, 1, 0, 0, 1, 0, 1], [0, 1, 1, 0, 0, 1, 0, 1]]
[[1, 0, 1, 1, 0, 1, 0, 0], [0, 0, 0, 0, 1, 1, 1, 1]]
[[0, 0, 1, 0, 0, 1, 1, 1], [0, 0, 1, 1, 0, 1, 1, 0]]
[[1, 1, 0, 1, 0, 1, 0, 0], [1, 1, 0, 1, 0, 1, 0, 0]]
[[0, 0, 1, 0, 1, 0, 0, 1], [0, 0, 1, 1, 0, 1, 1, 0]]


In [14]:
def genetic_algorithm(objective, n_bits, n_iter, n_pop, r_cross, r_mut):
    # initial population of random bitstring
    pop = [[randint(0,1) for x in range(n_bits)] for _ in range(n_pop)]
    print(pop)
    # keep track of best solution
    best, best_eval = 0, objective(pop[0])
    # enumerate generations
    for gen in range(n_iter):
        # evaluate all candidates in the population
        scores = [objective(c) for c in pop]
        # check for new best solution
        for i in range(n_pop):
            if scores[i] < best_eval:
                best, best_eval = pop[i], scores[i]
                print(">%d, new best f(%s) = %.3f" % (gen, pop[i], scores[i]))
        # select parents
        selected = [selection(pop, scores) for _ in range(n_pop)]
        # create the next generation
        children = list()
        for i in range(0, n_pop, 2):
            # get selected parents in pairs
            p1, p2 = selected[i], selected[i+1]
            # crossover and mutation
            for c in crossover(p1, p2, r_cross):
                # mutation
                mutation(c, r_mut)
                # store for next generation
                children.append(c)
        # replace population
        pop = children
    return [best, best_eval]

In [15]:
genetic_algorithm(objective, n_bits, n_iter, n_pop, r_cross, r_mut)

[[1, 0, 0, 0, 0, 1, 1, 0], [1, 0, 0, 0, 1, 0, 1, 1], [0, 1, 1, 0, 0, 0, 1, 1], [0, 0, 1, 0, 0, 1, 0, 0], [1, 1, 1, 0, 0, 1, 0, 1], [0, 0, 0, 1, 0, 0, 0, 0], [1, 0, 0, 0, 0, 1, 1, 0], [1, 1, 1, 1, 1, 0, 0, 1], [1, 1, 0, 1, 0, 0, 1, 0], [0, 0, 0, 1, 1, 1, 1, 0]]
>0, new best f([0, 0, 1, 0, 0, 1, 0, 0]) = 2.000
>0, new best f([0, 0, 0, 1, 0, 0, 0, 0]) = 1.000
[[1, 0, 0, 0, 0, 1, 1, 0], [0, 0, 0, 1, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 1, 1, 0], [0, 1, 1, 0, 0, 0, 1, 1]]
[[1, 0, 0, 0, 0, 1, 1, 0], [1, 0, 0, 0, 0, 1, 1, 0]]
[[0, 0, 1, 0, 0, 1, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 1, 1, 0], [0, 0, 0, 1, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0]]
[[0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 1, 1, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 1, 0, 1, 0, 0]]
[[0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 1, 1, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 1, 0, 1], [0, 0, 0, 1, 0, 0, 0, 0]]
>2, new best f([0, 0, 0, 0, 0, 0, 0, 0]) = 0.000
[[0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0

[[0, 0, 0, 0, 0, 0, 0, 0], 0]